# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'jobchangehyperml'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [3]:
cpu_cluster = "cpu-cluster2"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D12_V2',
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [32]:
# Parameter sampler
ps = RandomParameterSampling({
        "--C": choice(1,2,3,4),  #choice(1,2,3,4)
        "--max_iter": choice(10, 25, 50, 75)
    })

# Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2)

# SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', 
              compute_target = cpu_cluster, 
              entry_script = 'train.py')

# HyperDriveConfig using hyperparameter sampler, policy and estimator.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                hyperparameter_sampling = ps,
                                policy = policy,
                                primary_metric_name = 'Accuracy',
                                primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs = 20,
                                max_concurrent_runs = 4)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [35]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config) #, show_output=True
RunDetails(hyperdrive_run).show()
#hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6a22066b-0baf-42b2-bbd5-71cce78ef2d1
Web View: https://ml.azure.com/experiments/jobchangehyperml/runs/HD_6a22066b-0baf-42b2-bbd5-71cce78ef2d1?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-136548/workspaces/quick-starts-ws-136548

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-30T05:28:37.735023][API][INFO]Experiment created<END>\n"<START>[2021-01-30T05:28:38.5205182Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-30T05:28:38.588241][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-30T05:28:38.720090][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)
print('\n Accuracy: ', best_run_metrics['Accuracy'])

In [30]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
jobchangehyperml,HD_f2c90fc8-cc3f-4cc5-9d85-09c449125db1_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')


In [15]:
#Register the best model
best_model_registered = best_run.register_model(model_name = 'best_hyperdrive_model', 
                                                model_path='outputs/model.joblib',
                                                description='best hyperdrive model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
env = best_run.get_environment()

script_file = 'score.py'

#best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

In [22]:
best_run.properties

{'_azureml.ComputeTargetType': 'amlcompute',
 'ContentSnapshotId': '51fa46a9-31d3-4348-bd85-b35473782ed0',
 'ProcessInfoFile': 'azureml-logs/process_info.json',
 'ProcessStatusFile': 'azureml-logs/process_status.json'}

In [28]:
#Register the fitted model
#model_name = best_run.properties['model_name']
model = hyperdrive_run.register_model(model_name = 'HD_f2c90fc8-cc3f-4cc5-9d85-09c449125db1_4', #AutoMLac80c775a36
                                  description = 'AutoML model')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path HD_f2c90fc8-cc3f-4cc5-9d85-09c449125db1_4 in the set of files uploaded to the run: ['azureml-logs/hyperdrive.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path HD_f2c90fc8-cc3f-4cc5-9d85-09c449125db1_4 in the set of files uploaded to the run: ['azureml-logs/hyperdrive.txt']\n                See https://aka.ms/run-logging for more details."
    }
}

In [25]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'hyperml-job-change'
print(aci_service_name)

hyperml-job-change


In [ ]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
data_train = Dataset.get_by_name(ws, 'traindata')

In [26]:
output = service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

NameError: name 'service' is not defined

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

In [ ]:
service.delete()